In [ ]:
!pip install geopy
!pip install geocoder

In [2]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geocoder
import tqdm as tqdm
from tqdm.auto import tqdm
tqdm.pandas()
import time
import sys
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded
import time

In [3]:
rds = os.path.join('Resources','accident-data.csv')
rds_df = pd.read_csv(rds)
rds_df.head()

,accident_index,accident_year,accident_reference,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,...,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area
0,2020010219808,2020,10219808,-0.254001,51.462262,3,1,1,04/02/2020,3,...,6,0,9,9,1,9,9,0,0,1
1,2020010220496,2020,10220496,-0.139253,51.470327,3,1,2,27/04/2020,2,...,6,0,0,4,1,1,1,0,0,1
2,2020010228005,2020,10228005,-0.178719,51.529614,3,1,1,01/01/2020,4,...,6,0,0,0,4,1,2,0,0,1
3,2020010228006,2020,10228006,-0.001683,51.541210,2,1,1,01/01/2020,4,...,6,0,0,4,4,1,1,0,0,1
4,2020010228011,2020,10228011,-0.137592,51.515704,3,1,2,01/01/2020,4,...,5,0,0,0,4,1,1,0,0,1


In [4]:
rds_df['latitude'].isnull().values.sum()

14

In [5]:
rds_df.loc[rds_df['longitude'].isnull(), 'longitude'] = 0
rds_df.loc[rds_df['latitude'].isnull(), 'latitude'] = 0

In [6]:
def geo_rev(x):
    g = geocoder.osm([x.latitude, x.longitude], method='reverse').json
    time.sleep(1.1)
    for index, row in rds_df.iterrows():
        sys.stdout.write(f'\r{index}/{len(ndf)}')
        sys.stdout.flush()
        if g:
            return g.get('city')
        else:
            return 'no city'

#rds_df['city'] = rds_df[['latitude', 'longitude']].apply(geo_rev, axis=1)

In [7]:
ndf = rds_df.iloc[0:91199, 3:5]
ndf['geom'] = ndf["latitude"].map(str) + ',' + ndf["longitude"].map(str)
ndf

,longitude,latitude,geom
0,-0.254001,51.462262,"51.462262,-0.254001"
1,-0.139253,51.470327,"51.470327,-0.139253"
2,-0.178719,51.529614,"51.529614,-0.178719"
3,-0.001683,51.541210,"51.54121,-0.001683"
4,-0.137592,51.515704,"51.515704,-0.137592"
...,...,...,...
91194,-2.926320,56.473539,"56.473539,-2.92632"
91195,-4.267565,55.802353,"55.802353,-4.267565"
91196,-2.271903,57.186317,"57.186317,-2.271903"
91197,-3.968753,55.950940,"55.95094,-3.968753"


In [15]:
ndf1 = ndf.iloc[0:10000, 0:5]
ndf2 = ndf.iloc[10000:20000, 0:5]
ndf3 = ndf.iloc[20000:30000, 0:5]
ndf4 = ndf.iloc[30000:40000, 0:5]
ndf5 = ndf.iloc[40000:50000, 0:5]
ndf6 = ndf.iloc[50000:60000, 0:5]
ndf7 = ndf.iloc[60000:70000, 0:5]
ndf8 = ndf.iloc[70000:80000, 0:5]
ndf9 = ndf.iloc[80000:90000, 0:5]
ndf10 = ndf.iloc[90000:91199, 0:5]

In [16]:
ndf['city'] = ndf1[['latitude', 'longitude']].apply(geo_rev, axis=1)

0/91199

Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=51.461393%2C+-0.138421&format=jsonv2&addressdetails=1&limit=1 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))


0/91199

Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5.0)


0/91199

KeyboardInterrupt: 

In [ ]:
ndf

In [9]:
locator = Nominatim(user_agent='myGeocoder', timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [10]:
def geocode_me(location):
    time.sleep(1.1)
    try:
        return locator.geocode(location)
    except (GeocoderTimedOut, GeocoderQuotaExceeded) as e:
        if GeocoderQuotaExceeded:
            print(e)
        else:
            print(f'Location not found: {e}')
            return None


In [11]:
tqdm.pandas()
ndf['city'] = ndf1['geom'].progress_apply(geocode_me)
ndf.head()

  0%|          | 0/10000 [00:00<?, ?it/s]

GeocoderInsufficientPrivileges: Non-successful status code 403

In [ ]:
tqdm.pandas()
ndf['city'] = ndf2['geom'].progress_apply(rgeocode)
ndf.head()

In [ ]:
pd.rds_df.to_csv()